In [45]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [46]:
# Configuration
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use("ggplot")
from textwrap import wrap
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import Counter
import string
import nltk
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D

In [47]:
import pandas as pd
df = pd.read_csv('/home/yadagiri/Downloads/PAN HC3 CONCATENATED DATA/Raw_data_pre_processed.csv')
#df.to_csv('testing.csv', index=False)

# Human is 0 and AI is 1
# Press Ctrl + F9 to run this whole together

In [48]:
df

,text,label
0,Inaugural Address by President Joseph R. Biden...,0
1,Fact check: Biden inauguration impacted by pan...,0
2,Highlights from Joe Biden's 2021 inauguration\...,0
3,"Biden takes the helm, appeals for unity to tak...",0
4,'The Hill We Climb': Read Amanda Gorman's inau...,0
...,...,...
63829,"[""It's not uncommon for blood pressure to fluc...",1
63830,['There are several possible causes of a painl...,1
63831,"[""It is not appropriate for me to recommend a ...",1
63832,['It is not uncommon for people with rheumatoi...,1


In [5]:
# Removing 'id' and 'Category' columns
df = df.drop(columns=['model','source','id'])

In [66]:
df = df.rename(columns={'sentence': 'text','class': 'generated'})

In [6]:
df

,text,label
0,Giving gifts should always be enjoyable. Howe...,1
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1
2,If you'd rather not annoy others by being rude...,1
3,If you're interested in visiting gravesite(s) ...,1
4,The following are some tips for becoming succe...,1
...,...,...
4995,The paper deals with an interesting applicatio...,0
4996,This manuscript tries to tackle neural network...,0
4997,The paper introduced a regularization scheme t...,0
4998,Inspired by the analysis on the effect of the ...,0


In [5]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

def calculate(raw_df):
  def tokenize_text(text):
      if not isinstance(text, str):
          return []
      tokens = word_tokenize(text)
      return tokens

  def tokenize_sent(text):
      if not isinstance(text, str):
          return []
      sent = sent_tokenize(text)
      sent_sizes = [len(sent.split()) for sent in sent]
      return sent_sizes

  def sent_wala(text):
    if not isinstance(text, str):
          return []
    sent = sent_tokenize(text)
    return sent

  def vocab_for_row(text):
      unique=set(text)
      return len(unique)

  def number_of_sentence(text):
      return len(text)

  def number_of_words(text):
      return len(text)

  raw_df["words"] = raw_df["text"].apply(tokenize_text)

  raw_df["sentences"] = raw_df["text"].apply(sent_wala)

  raw_df["sentence lengths"] = raw_df["text"].apply(tokenize_sent)

  raw_df["vocab"] =raw_df["words"].apply(vocab_for_row)

  raw_df['line count'] = raw_df["sentence lengths"].apply(number_of_sentence)

  raw_df['word count'] = raw_df['words'].apply(number_of_words)

  raw_df['avg line length'] = raw_df["word count"]/raw_df["line count"]

calculate(df)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yadagiri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/yadagiri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Abhi mere paas 2 dataframe hai human_df and ai_df. Iske baad mujhe dono ko alag alag seh word density, average line length, and vocabulary count nikala hai.

In [6]:
def total_voc(df):
  all_words = [word for sublist in df['words'] for word in sublist]
  all_words = set(all_words)
  return len(all_words)


def sum_of_lines(df):
    return df['line count'].sum()

def average_line_len(df):
    sum_line_avg = df['avg line length'].sum()
    return sum_line_avg/ len(df)

def word_density(df):
    wd= 100*total_voc(df)/ (average_line_len(df)*sum_of_lines(df))
    return wd

In [7]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

def count_stop_words(words):
    doc = nlp(" ".join(words))
    stop_word_count = sum(1 for token in doc if token.is_stop)
    return stop_word_count

df["stopwords_count"]=df["words"].apply(count_stop_words)
df["stopwords_per"]=df["words"].apply(count_stop_words)/df["word count"]*100
df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,56.727829
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,47.193878
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,45.263158
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,49.140893
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,61.255116
...,...,...,...,...,...,...,...,...,...,...,...
63829,its not uncommon for blood pressure to fluctua...,1,"[its, not, uncommon, for, blood, pressure, to,...",[its not uncommon for blood pressure to fluctu...,[145],80,1,145,145.0,76,52.413793
63830,there are several possible causes of a painles...,1,"[there, are, several, possible, causes, of, a,...",[there are several possible causes of a painle...,[166],92,1,166,166.0,85,51.204819
63831,it is not appropriate for me to recommend a sp...,1,"[it, is, not, appropriate, for, me, to, recomm...",[it is not appropriate for me to recommend a s...,[146],79,1,146,146.0,78,53.424658
63832,it is not uncommon for people with rheumatoid ...,1,"[it, is, not, uncommon, for, people, with, rhe...",[it is not uncommon for people with rheumatoid...,[123],83,1,123,123.0,65,52.845528


In [8]:
# this is active and passive voice which i counted


import spacy
import pandas as pd


nlp = spacy.load("en_core_web_sm")


def is_passive(sentence):
    for token in sentence:
        if token.dep_ == "nsubjpass":
            return True
    return False

def count_active(sentences):
    active_count = 0
    passive_count = 0
    for sent in sentences:
        doc = nlp(sent)
        if is_passive(doc):
            passive_count += 1
    active_count=len(sentences)-passive_count
    return active_count


df["active"]=df["sentences"].apply(count_active)
df["passive"]=df["line count"]-df["active"]

df["active_per"]=df["active"]/df["line count"]*100
df["passive_per"]=df["passive"]/df["line count"]*100


Abhi Maine yaha peh POS TAGS nikale hai

In [9]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter

def count_nouns_verbs(word_list):
    tags = pos_tag(word_list)
    counts = Counter(tag for word, tag in tags)
    pos_counts = {
        'NOUN': counts.get('NN', 0) + counts.get('NNS', 0) + counts.get('NNP', 0) + counts.get('NNPS', 0),
        'VERB': counts.get('VB', 0) + counts.get('VBD', 0) + counts.get('VBG', 0) + counts.get('VBN', 0) + counts.get('VBP', 0) + counts.get('VBZ', 0),
        'PUNCT': counts.get('.', 0) + counts.get(',', 0) + counts.get(':', 0) + counts.get('(', 0) + counts.get(')', 0) + counts.get('"', 0) + counts.get("''", 0) + counts.get('``', 0) + counts.get('!', 0) + counts.get('?', 0) + counts.get(';', 0) + counts.get('-', 0),
        'DET': counts.get('DT', 0) + counts.get('PDT', 0) + counts.get('WDT', 0),
        'PRON': counts.get('PRP', 0) + counts.get('PRP$', 0),
        'PROPN': counts.get('NNP', 0) + counts.get('NNPS', 0),
        'ADJ': counts.get('JJ', 0) + counts.get('JJR', 0) + counts.get('JJS', 0),
        'AUX': counts.get('MD', 0),
        'ADV': counts.get('RB', 0) + counts.get('RBR', 0) + counts.get('RBS', 0),
        'PART': counts.get('RP', 0),
        'SCONJ': counts.get('IN', 0),
        'NUM': counts.get('CD', 0),
        'X': counts.get('FW', 0),
        'INTJ': counts.get('UH', 0),
        'ADP': counts.get('IN', 0),
        'SYM': counts.get('SYM', 0),
        'SPACE': counts.get('SP', 0),
        'CCONJ': counts.get('CC', 0)
    }
    return pd.Series(pos_counts)

pos_wala_df  = df['words'].apply(lambda x: pd.Series(count_nouns_verbs(x)))

human_copy=df[df['label'] == 0].copy()
ai_copy=df[df['label'] == 0].copy()

df = pd.concat([df, pos_wala_df], axis=1)
df


,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,...,34,1,88,5,0,0,88,0,0,28
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,...,18,0,48,1,0,0,48,0,0,9
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,...,21,1,76,5,0,0,76,0,0,19
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,...,27,2,77,2,0,0,77,0,0,23
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,...,55,3,78,6,0,0,78,0,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63829,its not uncommon for blood pressure to fluctua...,1,"[its, not, uncommon, for, blood, pressure, to,...",[its not uncommon for blood pressure to fluctu...,[145],80,1,145,145.0,76,...,8,0,13,1,0,0,13,0,0,5
63830,there are several possible causes of a painles...,1,"[there, are, several, possible, causes, of, a,...",[there are several possible causes of a painle...,[166],92,1,166,166.0,85,...,7,0,16,0,0,0,16,0,0,9
63831,it is not appropriate for me to recommend a sp...,1,"[it, is, not, appropriate, for, me, to, recomm...",[it is not appropriate for me to recommend a s...,[146],79,1,146,146.0,78,...,6,0,11,0,0,0,11,0,0,7
63832,it is not uncommon for people with rheumatoid ...,1,"[it, is, not, uncommon, for, people, with, rhe...",[it is not uncommon for people with rheumatoid...,[123],83,1,123,123.0,65,...,5,0,16,1,0,0,16,0,0,9


In [10]:
df.to_csv('/home/yadagiri/Downloads/new_data_hc3.csv', index=False)

In [32]:
human_df = df[df['label'] == 0].reset_index(drop=True)
ai_df = df[df['label'] == 1].reset_index(drop=True)

In [11]:
# for ai pos
ai_word_count=ai_df.iloc[:,7].sum()
pos_sum_ai = ai_df.iloc[:, 15:].sum()
pos_df_ai = pd.DataFrame(pos_sum_ai, columns=['POS_Sum'])
def contribution(val):
  return val*100/ai_word_count
pos_df_ai=pos_df_ai.apply(contribution)

# for human pos
human_word_count=human_df.iloc[:,7].sum()
pos_sum_human = human_df.iloc[:, 15:].sum()
pos_df_human = pd.DataFrame(pos_sum_human, columns=['POS_Sum'])
def contribution(val):
  return val*100/human_word_count
pos_df_human=pos_df_human.apply(contribution)

In [12]:
pos_df_ai
# ai contribution of pos tags

,POS_Sum
NOUN,29.556352
VERB,18.008841
PUNCT,0.000058
DET,11.765451
PRON,4.443503
PROPN,0.043046
ADJ,9.976871
AUX,1.512561
ADV,3.368631
PART,0.237867


In [13]:
pos_df_human
# human contribution of pos tags

,POS_Sum
NOUN,24.824978
VERB,19.105714
PUNCT,0.000393
DET,11.482003
PRON,6.298519
PROPN,0.035590
ADJ,9.431454
AUX,1.692409
ADV,6.259826
PART,0.524427


In [14]:
print(f"For Human :\nAverage Line Length {round(average_line_len(human_df),1)}\nTotal Vocabulary {total_voc(human_df)}\nWord Density {round(word_density(human_df),2)}")

print(f"\nFor AI :\nAverage Line Length {round(average_line_len(ai_df),1)}\nTotal Vocabulary {total_voc(ai_df)}\nWord Density {round(word_density(ai_df),2)}")

For Human :
Average Line Length 290.5
Total Vocabulary 86910
Word Density 1.18

For AI :
Average Line Length 269.0
Total Vocabulary 96498
Word Density 0.94


In [35]:
linking_words = {'to', 'the', 'and', 'of', 'in', 'on', 'for', 'with', 'at', 'a', 'an'}
def count_punctuation_marks(text):
    punctuation_count = sum(1 for char in text if char in string.punctuation)
    return punctuation_count

def count_linking_words(text):
    linking_words_count = sum(1 for word in word_tokenize(text.lower()) if word in linking_words)
    return linking_words_count

df['punctuation_count'] = df['text'].apply(count_punctuation_marks)

df['linking_words_count'] = df['text'].apply(count_linking_words)

df['punc_per']=df['punctuation_count']/df['word count']*100
df['link_per']=df['linking_words_count']/df['word count']*100



print(df)

/tmp/ipykernel_812060/1307602496.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['punctuation_count'] = df['text'].apply(count_punctuation_marks)


                                                    text  label  \
0      inaugural address by president joseph r biden ...      0   
1      fact check biden inauguration impacted by pand...      0   
2      highlights from joe bidens inauguration washin...      0   
3      biden takes the helm appeals for unity to take...      0   
4      the hill we climb read amanda gormans inaugura...      0   
...                                                  ...    ...   
63829  its not uncommon for blood pressure to fluctua...      1   
63830  there are several possible causes of a painles...      1   
63831  it is not appropriate for me to recommend a sp...      1   
63832  it is not uncommon for people with rheumatoid ...      1   
63833  it is not uncommon to experience back pain esp...      1   

                                                   words  \
0      [inaugural, address, by, president, joseph, r,...   
1      [fact, check, biden, inauguration, impacted, b...   
2      [highlig

/tmp/ipykernel_812060/1307602496.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['linking_words_count'] = df['text'].apply(count_linking_words)
/tmp/ipykernel_812060/1307602496.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['punc_per']=df['punctuation_count']/df['word count']*100
/tmp/ipykernel_812060/1307602496.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [19]:
ai_df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,inaugural address president joseph r biden jr ...,1,"[inaugural, address, president, joseph, r, bid...",[inaugural address president joseph r biden jr...,[134],76,1,134,134.0,55,...,0,0,15,0,0,0,15,0,0,8
1,setting the record straight factchecking the i...,1,"[setting, the, record, straight, factchecking,...",[setting the record straight factchecking the ...,[93],68,1,93,93.0,27,...,0,0,6,0,0,0,6,0,0,5
2,joe biden takes the oath of office as th presi...,1,"[joe, biden, takes, the, oath, of, office, as,...",[joe biden takes the oath of office as th pres...,[154],89,1,154,154.0,66,...,0,0,26,0,0,0,26,0,0,6
3,joe biden takes oath as th president of united...,1,"[joe, biden, takes, oath, as, th, president, o...",[joe biden takes oath as th president of unite...,[128],76,1,128,128.0,49,...,1,2,18,0,0,0,18,0,0,5
4,amanda gormans inspiring poem celebrates hope ...,1,"[amanda, gormans, inspiring, poem, celebrates,...",[amanda gormans inspiring poem celebrates hope...,[124],73,1,124,124.0,43,...,2,0,15,0,0,0,15,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38420,its not uncommon for blood pressure to fluctua...,1,"[its, not, uncommon, for, blood, pressure, to,...",[its not uncommon for blood pressure to fluctu...,[145],80,1,145,145.0,76,...,8,0,13,1,0,0,13,0,0,5
38421,there are several possible causes of a painles...,1,"[there, are, several, possible, causes, of, a,...",[there are several possible causes of a painle...,[166],92,1,166,166.0,85,...,7,0,16,0,0,0,16,0,0,9
38422,it is not appropriate for me to recommend a sp...,1,"[it, is, not, appropriate, for, me, to, recomm...",[it is not appropriate for me to recommend a s...,[146],79,1,146,146.0,78,...,6,0,11,0,0,0,11,0,0,7
38423,it is not uncommon for people with rheumatoid ...,1,"[it, is, not, uncommon, for, people, with, rhe...",[it is not uncommon for people with rheumatoid...,[123],83,1,123,123.0,65,...,5,0,16,1,0,0,16,0,0,9


In [20]:
human_df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,...,34,1,88,5,0,0,88,0,0,28
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,...,18,0,48,1,0,0,48,0,0,9
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,...,21,1,76,5,0,0,76,0,0,19
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,...,27,2,77,2,0,0,77,0,0,23
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,...,55,3,78,6,0,0,78,0,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25404,hellowelcome and thank you for asking on hcmre...,0,"[hellowelcome, and, thank, you, for, asking, o...",[hellowelcome and thank you for asking on hcmr...,[140],94,1,140,140.0,61,...,4,0,15,0,0,0,15,0,0,4
25405,hi as per my surgical experience the issue sou...,0,"[hi, as, per, my, surgical, experience, the, i...",[hi as per my surgical experience the issue so...,[69],62,1,69,69.0,33,...,1,1,10,0,0,0,10,0,0,1
25406,although it is difficult to comment whether ac...,0,"[although, it, is, difficult, to, comment, whe...",[although it is difficult to comment whether a...,[78],57,1,78,78.0,42,...,6,0,9,0,0,0,9,0,0,3
25407,welcome and thank you for asking on hcm i have...,0,"[welcome, and, thank, you, for, asking, on, hc...",[welcome and thank you for asking on hcm i hav...,[41],39,1,41,41.0,18,...,2,0,4,0,0,0,4,0,0,2


In [21]:
ai_copy

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per,active,passive,active_per,passive_per
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,56.727829,0,1,0.0,100.0
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,47.193878,0,1,0.0,100.0
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,45.263158,0,1,0.0,100.0
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,49.140893,0,1,0.0,100.0
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,61.255116,0,1,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39507,hellowelcome and thank you for asking on hcmre...,0,"[hellowelcome, and, thank, you, for, asking, o...",[hellowelcome and thank you for asking on hcmr...,[140],94,1,140,140.0,61,43.571429,0,1,0.0,100.0
39508,hi as per my surgical experience the issue sou...,0,"[hi, as, per, my, surgical, experience, the, i...",[hi as per my surgical experience the issue so...,[69],62,1,69,69.0,33,47.826087,1,0,100.0,0.0
39509,although it is difficult to comment whether ac...,0,"[although, it, is, difficult, to, comment, whe...",[although it is difficult to comment whether a...,[78],57,1,78,78.0,42,53.846154,1,0,100.0,0.0
39510,welcome and thank you for asking on hcm i have...,0,"[welcome, and, thank, you, for, asking, on, hc...",[welcome and thank you for asking on hcm i hav...,[41],39,1,41,41.0,18,43.902439,0,1,0.0,100.0


In [22]:
human_copy

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per,active,passive,active_per,passive_per
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,56.727829,0,1,0.0,100.0
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,47.193878,0,1,0.0,100.0
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,45.263158,0,1,0.0,100.0
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,49.140893,0,1,0.0,100.0
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,61.255116,0,1,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39507,hellowelcome and thank you for asking on hcmre...,0,"[hellowelcome, and, thank, you, for, asking, o...",[hellowelcome and thank you for asking on hcmr...,[140],94,1,140,140.0,61,43.571429,0,1,0.0,100.0
39508,hi as per my surgical experience the issue sou...,0,"[hi, as, per, my, surgical, experience, the, i...",[hi as per my surgical experience the issue so...,[69],62,1,69,69.0,33,47.826087,1,0,100.0,0.0
39509,although it is difficult to comment whether ac...,0,"[although, it, is, difficult, to, comment, whe...",[although it is difficult to comment whether a...,[78],57,1,78,78.0,42,53.846154,1,0,100.0,0.0
39510,welcome and thank you for asking on hcm i have...,0,"[welcome, and, thank, you, for, asking, on, hc...",[welcome and thank you for asking on hcm i hav...,[41],39,1,41,41.0,18,43.902439,0,1,0.0,100.0


In [23]:

human_stopwords_avg=round(human_df["stopwords_count"].sum()/df["word count"].sum()*100,2)
ai_stopwords_avg=round(ai_df["stopwords_count"].sum()/df["word count"].sum()*100,2)
print(f"Human Stopwords Average : {human_stopwords_avg}     AI Stopwords Average : {ai_stopwords_avg}")

Human Stopwords Average : 22.56     AI Stopwords Average : 28.29


In [24]:

human_active_avg=round(human_df["active"].sum()/human_df["line count"].sum()*100,2)
human_passive_avg=round(human_df["passive"].sum()/human_df["line count"].sum()*100,2)

ai_active_avg=round(ai_df["active"].sum()/ai_df["line count"].sum()*100,2)
ai_passive_avg=round(ai_df["passive"].sum()/ai_df["line count"].sum()*100,2)

print(f"Human Active Average : {human_active_avg}\nHuman Passive Average : {human_passive_avg}")
print(f"\nAI Active Average : {ai_active_avg}\nAI Passive Average : {ai_passive_avg}")

Human Active Average : 26.84
Human Passive Average : 73.16

AI Active Average : 20.73
AI Passive Average : 79.27


In [25]:
df['word density']=100*(df['vocab'])/(df['line count']*df['avg line length'])
df

/tmp/ipykernel_812060/796320893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word density']=100*(df['vocab'])/(df['line count']*df['avg line length'])


,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,INTJ,ADP,SYM,SPACE,CCONJ,punctuation_count,linking_words_count,punc_per,link_per,word density
0,inaugural address by president joseph r biden ...,0,"[inaugural, address, by, president, joseph, r,...",[inaugural address by president joseph r biden...,[653],304,1,654,654.0,371,...,0,88,0,0,28,0,144,0.0,22.018349,46.483180
1,fact check biden inauguration impacted by pand...,0,"[fact, check, biden, inauguration, impacted, b...",[fact check biden inauguration impacted by pan...,[392],210,1,392,392.0,185,...,0,48,0,0,9,0,90,0.0,22.959184,53.571429
2,highlights from joe bidens inauguration washin...,0,"[highlights, from, joe, bidens, inauguration, ...",[highlights from joe bidens inauguration washi...,[570],266,1,570,570.0,258,...,0,76,0,0,19,0,117,0.0,20.526316,46.666667
3,biden takes the helm appeals for unity to take...,0,"[biden, takes, the, helm, appeals, for, unity,...",[biden takes the helm appeals for unity to tak...,[582],307,1,582,582.0,286,...,0,77,0,0,23,0,154,0.0,26.460481,52.749141
4,the hill we climb read amanda gormans inaugura...,0,"[the, hill, we, climb, read, amanda, gormans, ...",[the hill we climb read amanda gormans inaugur...,[733],352,1,733,733.0,449,...,0,78,0,0,37,0,123,0.0,16.780355,48.021828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63829,its not uncommon for blood pressure to fluctua...,1,"[its, not, uncommon, for, blood, pressure, to,...",[its not uncommon for blood pressure to fluctu...,[145],80,1,145,145.0,76,...,0,13,0,0,5,0,26,0.0,17.931034,55.172414
63830,there are several possible causes of a painles...,1,"[there, are, several, possible, causes, of, a,...",[there are several possible causes of a painle...,[166],92,1,166,166.0,85,...,0,16,0,0,9,0,44,0.0,26.506024,55.421687
63831,it is not appropriate for me to recommend a sp...,1,"[it, is, not, appropriate, for, me, to, recomm...",[it is not appropriate for me to recommend a s...,[146],79,1,146,146.0,78,...,0,11,0,0,7,0,33,0.0,22.602740,54.109589
63832,it is not uncommon for people with rheumatoid ...,1,"[it, is, not, uncommon, for, people, with, rhe...",[it is not uncommon for people with rheumatoid...,[123],83,1,123,123.0,65,...,0,16,0,0,9,0,29,0.0,23.577236,67.479675


In [26]:
df.to_csv('/home/yadagiri/Downloads/PANHC3Preprocessedfeatures.csv', index=False)

In [25]:

X = pd.DataFrame(df['word density'])
X['punc_per'] = df['punc_per']
X['link_per'] = df['link_per']

X['avg line length'] = df['avg line length']
X['vocab'] = df['vocab']
X['active_per'] = df['active_per']
X['passive_per'] = df['passive_per']
X['stopwords_per'] = df['stopwords_per']

X['NOUN'] = df['NOUN']
X['VERB'] = df['VERB']
X['PUNCT'] = df['PUNCT']
X['DET'] = df['DET']
X['PRON'] = df['PRON']
X['ADJ'] = df['ADJ']
X['AUX'] = df['AUX']
X['ADV'] = df['ADV']
X['PART'] = df['PART']
X['SCONJ'] = df['SCONJ']
X['NUM'] = df['NUM']
X['X'] = df['X']
X['INTJ'] = df['INTJ']
X['ADP'] = df['ADP']
X['SYM'] = df['SYM']
X['SPACE'] = df['SPACE']
X['CCONJ'] = df['CCONJ']

y = df['generated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train_imputed, y_train)

y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
train_loss = log_loss(y_train, model.predict_proba(X_train_imputed))
test_loss = log_loss(y_test, model.predict_proba(X_test_imputed))
print("Train Loss:", train_loss)
print("Test Loss:", test_loss)

class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

class_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
print("Class-wise Metrics:")
print(class_report_df)

loss_values = [train_loss, test_loss]
labels = ['Training Loss', 'Testing Loss']

plt.figure(figsize=(8, 6))
plt.bar(labels, loss_values, color=['blue', 'orange'])
plt.title('Logistic Loss')
plt.ylabel('Loss')
plt.show()

KeyError: 'generated'

In [ ]:
X = pd.DataFrame(df['word density'])
X['punc_per'] = df['punc_per']
X['link_per'] = df['link_per']

X['avg line length'] = df['avg line length']
X['vocab'] = df['vocab']
X['active_per'] = df['active_per']
X['passive_per'] = df['passive_per']
X['stopwords_per'] = df['stopwords_per']

X['NOUN'] = df['NOUN']
X['VERB'] = df['VERB']
X['PUNCT'] = df['PUNCT']
X['DET'] = df['DET']
X['PRON'] = df['PRON']
X['ADJ'] = df['ADJ']
X['AUX'] = df['AUX']
X['ADV'] = df['ADV']
X['PART'] = df['PART']
X['SCONJ'] = df['SCONJ']
X['NUM'] = df['NUM']
X['X'] = df['X']
X['INTJ'] = df['INTJ']
X['ADP'] = df['ADP']
X['SYM'] = df['SYM']
X['SPACE'] = df['SPACE']
X['CCONJ'] = df['CCONJ']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='constant', fill_value=0)

X_train_imputed = imputer.fit_transform(X_train)
X_valid_imputed = imputer.transform(X_valid)
X_train_cnn = X_train_imputed.reshape(X_train_imputed.shape[0], X_train_imputed.shape[1], 1)
X_valid_cnn = X_valid_imputed.reshape(X_valid_imputed.shape[0], X_valid_imputed.shape[1], 1)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_data=(X_valid_cnn, y_valid))

y_train_pred_probs = model.predict(X_train_cnn)
y_valid_pred_probs = model.predict(X_valid_cnn)

y_train_pred = (y_train_pred_probs > 0.5).astype(int)
y_valid_pred = (y_valid_pred_probs > 0.5).astype(int)

train_error = 1 - accuracy_score(y_train, y_train_pred)

validation_error = 1 - accuracy_score(y_valid, y_valid_pred)
a
print("Train Error:", train_error)
print("Validation Error:", validation_error)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['human', 'AI'], yticklabels=['human', 'AI'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()